In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 01e3579b-7581-449e-8a3d-99e29b8d1b44, 53, Finished, Available, Finished)

In [ ]:
%%sql
create table if not exists Gold_Shipmode
(
    ShipMode_ID Long,
    Ship_Mode string,
    Created_TS timestamp,
    Modified_TS timestamp
)

StatementMeta(, 01e3579b-7581-449e-8a3d-99e29b8d1b44, 54, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [ ]:
Max_Date = spark.sql("select coalesce(max(Modified_TS),'1900-01-01') from Gold_Shipmode").first()[0]

StatementMeta(, 01e3579b-7581-449e-8a3d-99e29b8d1b44, 55, Finished, Available, Finished)

In [ ]:
Max_Date

StatementMeta(, 01e3579b-7581-449e-8a3d-99e29b8d1b44, 56, Finished, Available, Finished)

'1900-01-01'

In [ ]:
df_bronze = spark.sql("select distinct Ship_Mode from Bronze_Sales where Modified_TS >'{}'".format(Max_Date)) # we always catch the latest data

StatementMeta(, 01e3579b-7581-449e-8a3d-99e29b8d1b44, 57, Finished, Available, Finished)

In [ ]:
df_bronze.show()

StatementMeta(, 01e3579b-7581-449e-8a3d-99e29b8d1b44, 58, Finished, Available, Finished)

+--------------+
|     Ship_Mode|
+--------------+
|   First Class|
|      Same Day|
|  Second Class|
|Standard Class|
+--------------+



Now we will create incremental column, created_ts and modified_ts

for incremntal column creation we start by picking max value from ShipMode_ID from gold_shipmode table, first time it will be 0

In [ ]:
Max_ID = spark.sql("select coalesce(max(ShipMode_ID),0) from Gold_ShipMode").first()[0]

StatementMeta(, 01e3579b-7581-449e-8a3d-99e29b8d1b44, 59, Finished, Available, Finished)

In [ ]:
df_final = df_bronze.withColumn("ShipMode_ID",monotonically_increasing_id()+Max_ID+1)
df_final.createOrReplaceTempView("ViewShipMode")

StatementMeta(, 01e3579b-7581-449e-8a3d-99e29b8d1b44, 60, Finished, Available, Finished)

In [ ]:
%%sql
select * from ViewShipMode

StatementMeta(, 01e3579b-7581-449e-8a3d-99e29b8d1b44, 61, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 2 fields>

In [ ]:
%%sql
merge into Gold_ShipMode as GS
using ViewShipMode as VS 
on GS.Ship_Mode = VS.Ship_Mode
when matched then
update set 
Modified_TS = current_timestamp()

when not matched then
INSERT
(
GS.ShipMode_ID,
GS.Ship_Mode,
GS.Created_TS,
GS.Modified_TS
)
VALUES
(
VS.ShipMode_ID,
VS.Ship_Mode,
current_timestamp(),
current_timestamp()
)

StatementMeta(, 01e3579b-7581-449e-8a3d-99e29b8d1b44, 62, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>

In [ ]:
%%sql
select * from Gold_Shipmode

StatementMeta(, 01e3579b-7581-449e-8a3d-99e29b8d1b44, 63, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 4 fields>